
# FTC ML Workshop: From Data to Model (with TFLite)

This notebook is designed for **FTC teams** who want to understand **how Machine Learning works** instead of treating it as a black box. You'll:
- Explore and analyze data
- Train and evaluate models
- Understand common pitfalls (overfitting, data imbalance, leakage)
- Export a lightweight **TensorFlow Lite (TFLite)** model
- See how this maps to **FTC workflows** (VisionPortal + TFOD / FTC-ML)


## 0) Setup

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow.keras import layers, models

print("numpy", np.__version__)
print("pandas", pd.__version__)
import sklearn
print("scikit-learn", sklearn.__version__)
print("tensorflow", tf.__version__)



## 1) The ML Process
1. Define the problem
2. Collect and label data
3. Split into train/test
4. Train a model
5. Evaluate on test set
6. Iterate to improve
7. Export deployable model (.tflite for FTC)
8. Deploy & monitor


## 2) Tabular Example: Iris dataset

In [ ]:

iris = load_iris(as_frame=True)
df = iris.frame.copy()
df.head()


In [ ]:

df.describe()


In [ ]:

X = df[iris.feature_names].values
y = df['target'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

logreg_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000))
])
logreg_pipeline.fit(X_train, y_train)
y_pred = logreg_pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
plt.show()


In [ ]:

rf_pipeline = Pipeline([("clf", RandomForestClassifier(n_estimators=200, random_state=42))])
rf_pipeline.fit(X_train, y_train)
y_pred_rf = rf_pipeline.predict(X_test)
print("RF Accuracy:", accuracy_score(y_test, y_pred_rf))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_rf)
plt.show()


## 3) Vision Example: MNIST → TFLite

In [ ]:

(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = tf.keras.datasets.mnist.load_data()
mnist_x_train = mnist_x_train.astype("float32")/255.0
mnist_x_test = mnist_x_test.astype("float32")/255.0
mnist_x_train = np.expand_dims(mnist_x_train, -1)
mnist_x_test = np.expand_dims(mnist_x_test, -1)

model = models.Sequential([
    layers.Input(shape=(28,28,1)),
    layers.Conv2D(16,3,activation="relu"), layers.MaxPooling2D(),
    layers.Conv2D(32,3,activation="relu"), layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64,activation="relu"),
    layers.Dense(10,activation="softmax")
])
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(mnist_x_train, mnist_y_train, validation_split=0.1, epochs=3, batch_size=128, verbose=1)
print("Test acc:", model.evaluate(mnist_x_test, mnist_y_test, verbose=0))


In [ ]:

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("mnist_cnn.tflite","wb") as f: f.write(tflite_model)
print("Saved mnist_cnn.tflite, size:", len(tflite_model))



---
## Appendix: FTC‑ML 300×300 Helper

FTC‑ML resizes all training images to **300×300**.  
This helper function letterboxes any image to that size while preserving aspect ratio.


In [ ]:

import cv2, numpy as np

def letterbox_300(img):
    H, W = img.shape[:2]
    scale = min(300/W, 300/H)
    nw, nh = int(round(W*scale)), int(round(H*scale))
    resized = cv2.resize(img, (nw, nh), interpolation=cv2.INTER_AREA)
    canvas = np.zeros((300,300,3), dtype=img.dtype)
    x0 = (300-nw)//2; y0=(300-nh)//2
    canvas[y0:y0+nh, x0:x0+nw] = resized
    return canvas, scale, x0, y0
